<a href="https://colab.research.google.com/github/krishnamalani1164/xai-intrusion-detection-shap/blob/main/xai_intrusion_detection_shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Overview
This project demonstrates the application of Explainable AI techniques to security applications,
specifically focusing on Network Intrusion Detection Systems (IDS). The goal is to create
machine learning models that can effectively detect network intrusions while providing
transparent explanations for their decisions.

## Key Components
1. Data Processing: Using the NSL-KDD dataset, a benchmark dataset for intrusion detection
2. Model Training: Implementation of XGBoost classifier for attack detection
3. Explainable AI: Application of SHAP and LIME techniques to explain model predictions
4. Security Analysis: Evaluation of model performance and robustness against adversarial inputs

#Installation of Dependicies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import warnings
import shap
warnings.filterwarnings("ignore")

#Assigning column names

In [2]:
from google.colab import files
uploaded = files.upload()

Saving NSL-KDD  Dataset.zip to NSL-KDD  Dataset.zip


# Step 2: Unzip the file

In [3]:
import zipfile
import os

zip_path = next(iter(uploaded))  # Get the uploaded zip filename
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()  # Extract into current working directory

In [4]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [9]:
# Assigning attribute name to dataset
df_train = pd.read_csv("KDDTrain+.txt", header=None, names=col_names)
df_test = pd.read_csv("KDDTest+.txt", header=None, names=col_names)

# Step 5: Display a few rows to confirm
print("Train Dataset:")
print(df_train.head())

print("\nTest Dataset:")
print(df_test.head())

Train Dataset:
  duration protocol_type service  flag  src_bytes  dst_bytes  land  \
0      tcp      ftp_data      SF   491          0          0     0   
0      udp         other      SF   146          0          0     0   
0      tcp       private      S0     0          0          0     0   
0      tcp          http      SF   232       8153          0     0   
0      tcp          http      SF   199        420          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                0.17   
0               0       0    0  ...                0.00   
0               0       0    0  ...                0.10   
0               0       0    0  ...                1.00   
0               0       0    0  ...                1.00   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.03                    0.17   
0                    0.60                    0.88   
0                    0.05                    0.00 

# 2. DATA EXPLORATION AND PREPROCESSING


In [10]:
print("\n[2] DATA EXPLORATION AND PREPROCESSING")


[2] DATA EXPLORATION AND PREPROCESSING


In [12]:
print("\n Sample data from training set:")
print(df_train.head())


 Sample data from training set:
  duration protocol_type service  flag  src_bytes  dst_bytes  land  \
0      tcp      ftp_data      SF   491          0          0     0   
0      udp         other      SF   146          0          0     0   
0      tcp       private      S0     0          0          0     0   
0      tcp          http      SF   232       8153          0     0   
0      tcp          http      SF   199        420          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0               0       0    0  ...                0.17   
0               0       0    0  ...                0.00   
0               0       0    0  ...                0.10   
0               0       0    0  ...                1.00   
0               0       0    0  ...                1.00   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.03                    0.17   
0                    0.60                    0.88   
0                    0.05       

#Basic information about the dataset

In [13]:
print("\n sample data from missing values:")
print(df_train.info())


 sample data from missing values:
<class 'pandas.core.frame.DataFrame'>
Index: 125973 entries, 0 to 0
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  object 
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  int64  
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              1259

# Attack type distribution

In [14]:
print("\n Attack distribution in training_data:")
attack_counts = df_train['label'].value_counts()
print(attack_counts)


 Attack distribution in training_data:
label
21    62557
18    20667
20    19339
19    10284
15     3990
17     3074
16     2393
12      729
14      674
11      641
13      451
10      253
9       194
7       118
8       106
6        96
5        81
4        79
0        66
3        65
1        62
2        54
Name: count, dtype: int64


In [15]:
#Map detailed attack labels to attack categories
def categorize_attack(label):
  if label == 'normal':
    return 'normal'
  elif label in ['neptune', 'back', 'land', 'pod', 'smurf', 'teardrop', 'apache2', 'udpstorm', 'processtable', 'worm']:
      return 'DoS'
  elif label in ['satan', 'ipsweep', 'nmap', 'portsweep', 'mscan', 'saint']:
      return 'Probe'
  elif label in ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster', 'warezclient', 'spy', 'xlock', 'xsnoop', 'snmpguess', 'snmpgetattack', 'httptunnel', 'sendmail', 'named']:
      return 'R2L'
  elif label in ['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'sqlattack', 'xterm', 'ps']:
      return 'U2R'
  else:
      return 'Unknown'

In [16]:
#Apply categorization
df_train['attack_cat'] = df_train['label'].apply(categorize_attack)
df_test['attack_cat'] = df_test['label'].apply(categorize_attack)

In [17]:
# Display attack categories
print("\n  Attack categories in training data:")
print(df_train['attack_cat'].value_counts())


  Attack categories in training data:
attack_cat
Unknown    125973
Name: count, dtype: int64


In [ ]:
#Visualization of attack categories